<a href="https://colab.research.google.com/github/cleberjamaral/autoOrgDesignProject/blob/master/GoOrg4Prod_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
bound_df = pd.read_csv("Feed_production_line_binding.csv","\t")
skills = ["db_access","move","lift","pnp"]
#print(bound_df["pTree"])

In [3]:
def powerset(s):
    P = []
    x = len(s)
    for i in range(1 << x):
        P.append([s[j] for j in range(x) if (i & (1 << j))])
    return P

In [4]:
def get_organisations(array_str):
    organisations = []
    for i in range(0,len(array_str)):
        positions = []
        positions_str = array_str[i].split("G{")
        for j in range(0,len(positions_str)):
            position_str = positions_str[j].replace("[","").replace("]","").replace("}","").replace("^","").split(",")
            #print("p",j,position_str)
            position = []
            for k in range(0,len(position_str)):
                goal = position_str[k].strip()
                skill = goal.replace("FeedProduction","db_access").replace("MoveBox","move").replace("MoveBox$0","move").replace("MoveBox$1","move").replace("GetBox","lift").replace("PlaceBox","pnp")
                #print("g",k,goal)
                if len(goal)>0:
                    position.append(skill)
                #print(position)
            if len(position)>0:
                positions.append(position)
        #print(positions)
        organisations.append(positions)
    return organisations

In [5]:
# Test if all positions generated by GoOrg4Prod are indeed in the
# set of all_agent_kinds
def test_all_orgs_match_agents_power_set():
    all_match = True
    orgs = get_organisations(bound_df["pTree"])

    all_agent_kinds = powerset(skills)
    np.savetxt("all_agent_kinds.csv",all_agent_kinds,delimiter = "\t",fmt = "% s")

    #for i in range(0,len(all_agent_kinds)):
    #    print(all_agent_kinds[i])
    for o in range(0,len(orgs)):
        #print(o,orgs[o])   
        for p in orgs[o]:
            contains = False
            for k in all_agent_kinds:
                if list(set(p)) in [list(set(k))]:
                    #print(":::",list(set(p)),"isin",all_agent_kinds)
                    contains = True

            if not contains:
                print(">>>",list(set(p)),":notin:",all_agent_kinds)
                all_match = False

    return all_match
    

In [7]:
print("orgs match agents powerset?",test_all_orgs_match_agents_power_set())

orgs match agents powerset? True


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
# For each possible set of agents (from the list that contains 65536 sets) this procedure
# checks if the set is applicable to every generated structure (the list contains 1646 structures)
# It also checks the overqualification cost of the matches
def check_all_applicable_sets_of_agents():
    orgs = get_organisations(bound_df["pTree"])

    all_agent_kinds = powerset(skills)
    all_sets_of_agents = powerset(all_agent_kinds)

    applicables = []
    count_match = []
    all_match_sets = []
    count_zero_q_cost = []
    for sa in range(0,len(all_sets_of_agents)):

        if (sa % 200) == 0:
            print(sa+1,"of",len(all_sets_of_agents),"items processed.")
        # Sort set to make sure to test first the sets that are cheaper in terms of qualification cost
        sorted_set = sorted(all_sets_of_agents[sa], key = len)

        # Check if the set has all necessary skills, which is a condition to be applicable
        # If there is any skill missing, this set would not be relevant, no organisation will be feasible
        all_sets_of_agents_flat = [val for sublist in sorted_set for val in sublist]
        if all(elem in all_sets_of_agents_flat for elem in skills):
            applicables.append(1)
        else:
            applicables.append(0)

        orgs = get_organisations(bound_df["pTree"])
        matches = 0
        match_set = []
        zero_q_cost = 0
        for o in range(0,len(orgs)):
            all_match = True
            overqualification_cost = 0
            for p in orgs[o]:
                match = False
                for k in sorted_set:
                    if all(elem in set(k) for elem in set(p)):
                        match = True
                        overqualification_cost += len(set(k))-len(set(p))
                        break
                if not match:
                    all_match = False
                    break
            if all_match:
                matches += 1
                match_set.append([o+1,overqualification_cost])
                if (overqualification_cost == 0):
                    zero_q_cost += 1
        count_match.append(matches)
        all_match_sets.append(match_set)
        count_zero_q_cost.append(zero_q_cost)

        #if sa >= 500:
        #    break
        
    np.savetxt("all_sets_of_agents.csv",list(zip(all_sets_of_agents,applicables,count_match,all_match_sets,count_zero_q_cost)),delimiter = "\t",fmt = "% s")

In [9]:
check_all_applicable_sets_of_agents()

1 of 65536 items processed.
201 of 65536 items processed.
401 of 65536 items processed.


KeyboardInterrupt: ignored